In [39]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup

## 완결 웹툰 크롤링 ##

sel_sql = "select toon_id, is_end from toon_info where toon_id = %s"
ins_sql = "insert into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
upd_sql = "update toon_info set is_end = %s where toon_id = %s and is_end = %s"

soup = readHtml("https://comic.naver.com/webtoon/finish.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))
# pprint(a_list)

# 웹툰 정보 추출
count = 0
for a in a_list:
    src = a['href']
    url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    url = urlparse(src)
    toon_id = 'naver_' + str(parse_qs(url.query)['titleId']).replace("['", '').replace("']", '') # ID 생성

    toon_url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(toon_url+'1') # 네이버 웹툰 첫페이지
    
    detail = soup.find('div', {'class':'detail'})
    toon_name = detail.find('h2')
    
    wrt_nm = str(toon_name.find('span', {'class':'wrt_nm'}))
    wrt_name = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    toon_name = str(toon_name).replace('\t', '').replace('컷툰', '')
    toon_name = re.sub('<.+?>', '', toon_name, 0).strip()
    terminator = toon_name.index('\n')
    toon_name = toon_name[:terminator].strip('컷') # 제목 추출
    
    # 장르 추출
    genres = str(detail.find('span', {'class':'genre'}))
    genres = re.sub('<.+?>', '', genres, 0).strip()
    genre_list = genres.split(', ')

    desc = str(detail.find('p')).replace('<br/>', '\n')
    toon_desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    toon_thumb_url = soup.find('img')['src'] # 썸네일 주소 추출
    
    # toon_info
    cursor1.execute(sel_sql, toon_id)
    result = cursor1.fetchall()

    res_o = (item for item in result if (toon_id == item['toon_id'] and 'O' == item['is_end']))
    res_o_list = []
    for res_o_item in res_o:
        res_o_list.append(res_o_item)
        
    res_x = (item for item in result if (toon_id == item['toon_id'] and 'X' == item['is_end']))
    res_x_list = []
    for res_x_item in res_x:
        res_x_list.append(res_x_item)
    
    # 이미 완결 웹툰 DB에 들어있는지 확인
    if res_o_list:
        print('already exist')
        count += 1

    # 연재 중이었다가 완결이 난 웹툰의 완결 여부 수정
    elif res_x_list:
        cursor4.execute(upd_sql, ('O', toon_id, 'X'))
    
    else:
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'naver', wrt_name, 'O', toon_thumb_url))
        print('execute cur2')
        for genre_name in genre_list:
            cursor3.execute(genre_sql, (toon_id, genre_name))
            print('execute cur3')
            db.commit()
            
    if (count==10):
        print('finish commit')
        break
        
db.close()

already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
already exist
finish commit


1
1
2
2
3
